# Dependent Sample Generation

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import t, chi2
import pickle
from matplotlib.pylab import plt

from prisk.analysis_functions import (
    combine_glofas, 
    extract_discharge_timeseries, 
    fit_gumbel_distribution, 
    calculate_uniform_marginals)

In [2]:
glofas_dir = "/Users/rubenkerkhofs/Desktop/glofas/" 
basin_outlet_file = "https://kuleuven-prisk.s3.eu-central-1.amazonaws.com/lev06_outlets_final_clipped_Thailand_no_duplicates.csv"

In a first step, we obtain discharge data for the basins:

In [3]:
# Step 1: Load GloFAS river discharge data and upstream accumulating area data
# Discharge data for producing GIRI maps is from 1979-2016
start_year = 1979
end_year = 2016
area_filter = 500 # not considering rivers with upstream areas below 500 km^2
glofas_data = combine_glofas(start_year, end_year, glofas_dir, area_filter)

# Step 2: Load the basin outlet file, perform some data checks (to ensure we have valid discharge timeseries at each basin outlet point), and then extract discharge timeseries for each basin
basin_outlets = pd.read_csv(basin_outlet_file)
# Note to align the two datasets we need to make the following adjustment to lat lons (based on previous trial and error)
basin_outlets['Latitude'] = basin_outlets['Latitude'] + 0.05/2
basin_outlets['Longitude'] = basin_outlets['Longitude'] - 0.05/2
# Extract discharge timeseries
basin_timeseries = extract_discharge_timeseries(basin_outlets, glofas_data)

Once, the timeseries are obtained, we fit the gumbel distribution to each individual basin:

In [4]:
gumbel_params, fit_quality = fit_gumbel_distribution(basin_timeseries)

Once the Gumbel distributions are fitted, we compute the uniform marginals:

In [5]:
uniform_marginals = calculate_uniform_marginals(basin_timeseries, gumbel_params)

These uniform marginals are used to estimate the dependency structure between basins.

### Gaussian Copula
The Gaussian copula only requires the correlation matrix as an input parameter. We use the GaussianMultivariate object of the copulas package to estimate and sample from this copula. Note that the GaussianMultivariate object also estimates the univariate distributions; however, we have already transformed the univariate distributions to the uniform distribution. For that reason, we fix the uniform distribution.

In [6]:
from copulas.multivariate import GaussianMultivariate
from copulas.univariate import UniformUnivariate

class UniformUnivariateFixed(UniformUnivariate):
    def _fit_constant(self, X):
        self._params = {
            'loc': 0,
            'scale': 1
        }

    def _fit(self, X):
        self._params = {
            'loc': 0,
            'scale': 1
        }

data = pd.DataFrame(uniform_marginals)
# Confusingly, the distribution parameter sets the marginals
copula = GaussianMultivariate(distribution=UniformUnivariateFixed)
copula.fit(data)

Then, the sample function can be used to obtain samples:

In [7]:
copula.sample(10)

,4060020990,4060019720,4060021260,4060021270,4060021320,4060019710,4060019420,4060021330,4060034180,4060019410,...,4061043000,4061041330,4061042900,4061040840,4061040790,4061041310,4061031450,4061031350,4061028990,4061028980
0,0.213675,0.152932,0.182754,0.227270,0.320394,0.072965,0.188934,0.940515,0.830858,0.718133,...,0.446101,0.664719,0.414697,0.478813,0.363942,0.371142,0.448829,0.351858,0.332125,0.492879
1,0.933594,0.827748,0.490363,0.532355,0.341569,0.688935,0.367078,0.436991,0.260301,0.619236,...,0.433579,0.101696,0.174660,0.305523,0.210877,0.207080,0.518470,0.203761,0.628224,0.368635
2,0.513718,0.988456,0.886224,0.972288,0.826014,0.976963,0.910688,0.647884,0.791788,0.455545,...,0.355239,0.808630,0.378956,0.867566,0.292668,0.335025,0.203024,0.308800,0.457554,0.622215
3,0.322622,0.638918,0.970848,0.991132,0.994552,0.663103,0.986381,0.967206,0.934202,0.885069,...,0.898962,0.488862,0.843085,0.525266,0.806198,0.822162,0.873538,0.775131,0.521089,0.985414
4,0.618747,0.857640,0.974757,0.957667,0.974255,0.927795,0.983311,0.697191,0.757508,0.917437,...,0.391613,0.713343,0.299782,0.770077,0.242754,0.272930,0.657547,0.220782,0.188137,0.370144
5,0.596331,0.678088,0.498860,0.625219,0.827933,0.626907,0.691138,0.743259,0.677971,0.536131,...,0.023522,0.432576,0.149020,0.124769,0.155463,0.141378,0.110101,0.177114,0.257324,0.988625
6,0.969824,0.792757,0.966313,0.943214,0.956912,0.731772,0.880846,0.942948,0.966692,0.939819,...,0.549344,0.849031,0.660539,0.315263,0.695465,0.683398,0.356219,0.714511,0.402051,0.326960
7,0.456704,0.458296,0.609245,0.296369,0.190697,0.625947,0.708396,0.156987,0.090183,0.616789,...,0.457659,0.768551,0.691998,0.382173,0.723146,0.686860,0.574920,0.709879,0.735056,0.813390
8,0.871946,0.651128,0.870675,0.886037,0.944775,0.807099,0.968465,0.755795,0.820200,0.731887,...,0.331050,0.627909,0.508419,0.624066,0.541042,0.510909,0.168489,0.602194,0.708508,0.742590
9,0.415632,0.886065,0.681489,0.815998,0.893355,0.715037,0.839815,0.959423,0.985893,0.860700,...,0.237181,0.510522,0.015616,0.458071,0.009535,0.013556,0.305014,0.011764,0.333003,0.154672


In [8]:
data.shape

(38, 120)

### T-Copula
The T-copula requires two model inputs: (1) the correlation matrix, and (2) the degrees of freedom. In this case, we set the degrees of freedom equal to 3. Samples of the T-Copula are obtained as follows:


In [9]:
n_samples = 5

corr_matrix = data.corr().values
mu = np.zeros(len(corr_matrix))
s = chi2.rvs(df=3, size=n_samples)[:, np.newaxis]
Z = np.random.multivariate_normal(mu, corr_matrix, n_samples)
X = np.sqrt(3/s)*Z
U = t.cdf(X, df=3)

t_samples = pd.DataFrame(U, columns=data.columns)

### Vine Copula
The vine copula is estimated using the vinecopulas package. The estimated parameters are pickled.

In [10]:
from vinecopulas.vinecopula import fit_vinecop

M, P, C = fit_vinecop(data.values, copsi=[10])

** Tree:  1
24,16  --->  Frank : parameters =  3.6995743927299305
1,5  --->  Frank : parameters =  9.059094505621841
16,15  --->  Frank : parameters =  6.927964455142342
0,5  --->  Frank : parameters =  4.421241287110731
5,6  --->  Frank : parameters =  4.416063329994757
15,13  --->  Frank : parameters =  5.493770763597069
3,2  --->  Frank : parameters =  11.03880909691623
11,9  --->  Frank : parameters =  14.147953433417676
30,32  --->  Frank : parameters =  7.629295793137233
2,6  --->  Frank : parameters =  8.54990144157313
36,35  --->  Frank : parameters =  10.21583977701177
6,4  --->  Frank : parameters =  7.34149449324023
13,10  --->  Frank : parameters =  12.619065503936314
32,29  --->  Frank : parameters =  6.692482677279797
4,8  --->  Frank : parameters =  7.001447798641107
35,45  --->  Frank : parameters =  5.259617146071521
45,50  --->  Frank : parameters =  8.166238889201434
29,37  --->  Frank : parameters =  6.9466032496929735
8,7  --->  Frank : parameters =  12.31850902938

In [11]:
from vinecopulas.vinecopula import sample_vinecop
from prisk.analysis_functions import calculate_basin_copula_pairs, minimax_ordering


samples = pd.DataFrame(sample_vinecop(M, P, C, 5), columns=data.columns)
#samples.to_parquet("vine_random_numbers_frank.parquet.gzip", compression="gzip", index=False)